In [89]:
# Keeping this class here for now (easy to test). Will move this to a separate .py file when we do code-cleanup/ submission

import psycopg2
import psycopg2.extras
import re
import pandas


class FoodViolationData:

    def __init__(self, connection_string):
        self.conn = psycopg2.connect(connection_string)

    class FoodViolationQuery:

        def __init__(self, product_data):
            self.joins = []
            self.conditions = []
            self.parameters = {}
            self.product_data = product_data

            self.named_params = {}

        def __param_index(self, param_name):

            if param_name not in self.named_params:
                self.named_params.update({param_name: 0})
                return 0
            else:
                i = self.named_params[param_name] + 1
                self.named_params.update({param_name: i})
                return i

        def max_calories(self, max_calories):
            self.conditions.append("products.calories_100g <= %(max_calories)s")
            self.parameters.update({"max_calories": max_calories})
            return self

        def max_sugar(self, max_sugar):
            self.conditions.append("products.sugars_100g <= %(max_sugar)s")
            self.parameters.update({"max_sugar": max_sugar})
            return self

        def ingredient(self, ingredient):
            i = str(self.__param_index("ingredient"))
            self.joins.append(
                "JOIN product_ingredient as product_ingredient" + i + " on products.code = product_ingredient" + i + ".product JOIN ingredients as ingredients" + i + " on ingredients" + i + ".name = product_ingredient" + i + ".ingredient")
            self.conditions.append("LOWER(ingredients" + i + ".name) like LOWER(%(ingredient" + i + ")s)")
            self.parameters.update({"ingredient" + i: "%" + ingredient + "%"})
            return self

        def brand(self, brand):
            i = str(self.__param_index("brand"))
            self.conditions.append("LOWER(brands) like LOWER(%(brand" + i + ")s)")
            self.parameters.update({"brand" + i: "%" + brand + "%"})
            return self

        def category(self, category):
            i = str(self.__param_index("category"))
            self.joins.append(
                "JOIN product_category as product_category" + i + " on products.code = product_category" + i + ".product JOIN categories as categories" + i + " on categories" + i + ".name = product_category" + i + ".category")
            self.conditions.append("LOWER(categories" + i + ".name) like LOWER(%(category" + i + ")s)")
            self.parameters.update({"category" + i: "%" + category + "%"})
            return self

        def name(self, name):
            self.conditions.append("LOWER(products.name) like LOWER(%(name)s)")
            self.parameters.update({"name": name})
            return self

        def searchByCounty(self, name):
            i = str(self.__param_index("county"))
            self.conditions.append(""" "COUNTY" like %(county""" + i + ")s")
            #print(i)
            #print(name)
            #self.conditions.append(""" "COUNTY" like '%%""" + name + """%%' """)
            #print(self.conditions)
            self.parameters.update({"county" + i: "%" + name + "%"})
            #print(self.parameters)
            return self

        def __compile(self):
            query = "SELECT * from food_service_inspections"
            query = query + " " + " ".join(self.joins) if self.joins else query
            query = query + " WHERE " + " AND ".join(self.conditions) if self.conditions else query
            #query += " ORDER BY food_service_inspections.code"
            return query

        def run(self):
            query = self.__compile()
            #query = """select * from food_service_inspections where "COUNTY" like 'ALB%%' limit 3"""
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                print(query)
                cursor.execute(query, self.parameters)
                return pandas.DataFrame(cursor.fetchall())

        def show(self):
            query = self.__compile()
            with self.product_data.conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cursor:
                return cursor.mogrify(query, self.parameters)

    def query(self):
        return self.FoodViolationQuery(self)

In [90]:
import re
import pandas
import matplotlib

In [91]:
fd_vio = FoodViolationData("host='localhost' dbname='test5' user='test5' password='test5'")

In [93]:
# Searching for rows with have "BANY" and "NAS" in their county names (expect 0 rows)
fd_vio.query().searchByCounty("BANY").searchByCounty("NAS").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s AND  "COUNTY" like %(county1)s


""


In [94]:
# Just searching for "BANY" in COUNTY names
fd_vio.query().searchByCounty("BANY").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s


,COUNTY,DATE OF INSPECTION,NYS HEALTH OPERATION ID,VIOLATION ITEM,CRITICAL VIOLATION,TOTAL # CRITICAL VIOLATIONS,TOTAL #CRIT. NOT CORRECTED,TOTAL # NONCRITICAL VIOLATIONS,NYSDOH GAZETTEER 1980,INSPECTION TYPE,INSPECTION COMMENTS
0,ALBANY,2/11/2009,687080,12E,Not Critical Violation,0.0,0.0,3.0,10300,Inspection,None
1,ALBANY,2/3/2014,692526,8A,Not Critical Violation,1.0,0.0,6.0,15300,Inspection,None
2,ALBANY,8/23/2017,692526,14A,Not Critical Violation,0.0,0.0,3.0,15300,Inspection,excellent 8/23/2017
3,ALBANY,5/24/2010,676591,8A,Not Critical Violation,2.0,0.0,5.0,15500,Inspection,None
4,ALBANY,6/21/2017,676591,8E,Not Critical Violation,0.0,0.0,6.0,15500,Inspection,None
5,ALBANY,2/9/2011,676591,8E,Not Critical Violation,0.0,0.0,3.0,15500,Inspection,None
6,ALBANY,9/10/2009,676591,9B,Not Critical Violation,1.0,0.0,7.0,15500,Inspection,None
7,ALBANY,4/2/2012,676591,11A,Not Critical Violation,3.0,0.0,8.0,15500,Inspection,None
8,ALBANY,2/23/2015,676591,15A,Not Critical Violation,2.0,0.0,6.0,15500,Inspection,None
9,ALBANY,4/23/2019,676591,6A,Critical Violation,1.0,0.0,2.0,15500,Inspection,"Excellent compliance posted April 23, 2019\n\n..."


In [96]:
#Searching 3 at time, should return 0 rows
fd_vio.query().searchByCounty("BANY").searchByCounty("NAS").searchByCounty("Test").run()

SELECT * from food_service_inspections WHERE  "COUNTY" like %(county0)s AND  "COUNTY" like %(county1)s AND  "COUNTY" like %(county2)s


""
